In [16]:
from aiModel import QwenModel
from utils import logger, execute_sql_query,table_details
from config import api_key_qwen
import time
import json

In [10]:
def call_model(llm, messages, max_retries=3):
    for i in range(max_retries):
        try:
            return llm.call(messages)
        except Exception as e:
            if i < max_retries - 1: 
                logger.error('在发送请求时发生错误: %s', e)
                time.sleep(2 ** i)
            else: 
                raise



In [12]:
def tables():
    table_info ='''
    "核心数据库表概览": {
    "tender_index": "汇总招标公告基本信息，涵盖公告名字、发布日期等关键字段，适合快速检索公告概览。",
    "tender_key_detail": "集中存储招标公告中的重要细节，如价格范围、工期、面积、总造价及具体要求，用于满足对招标核心条件的查询。",
    "tender_detail": "提供招标公告其他结构化信息补充，丰富tender_key_detail未涵盖的内容。",
    "tender_detail_html": "保存招标公告原文HTML及来源信息，便于查阅原始公告或分析网页结构。",
    "announcement_catalog": "公告类型分类，帮助按公告种类进行筛选。",
    "announcement_labels": "为公告提供标签，支持通过标签快速定位。",
    "company_qualification_type": "收录国家规定的资质类型官方名称列表，用于匹配用户提供的资质简称或模糊描述，提升查询准确性。"
    }'''
    return table_info


In [31]:
def QueryBuilder(user_input):
    tools = [
    {
    "type": "function",
    "function": {
        "name": "tables",
        "description": "返回数据库中所有表的名称及简要信息。",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        },
        }
    },
    {
    "type": "function",
    "function": {
        "name": "table_details",
        "description": "查询表的详细架构和前几行数据。",
        "parameters": {
            "type": "object",
            "properties": {
                "tables": {
                    "description": "表名列表，用于查询表的详细架构。如['table1', 'table2']",
                    "type": "list"
                }
            },
            "required": ["tables"]
        },
        }
    },

    ]

    role_setting = '''
    **职责**：
    - 接收用户问题。
    - 使用给定的Tools来分析问题，确定相关数据库表和字段。
    - 构建验证查询以获取表结构详情。
    - 根据验证结果，构建最终的SQL查询语句。

    **工作流程**:
    1. **理解问题**: 分析用户提问，识别可能涉及的业务领域和数据需求。
    2. **查询表信息**: 使用`tables` Tool 获取所有表的概览信息，以便初步筛选相关表。
    3. **获取表详情**: 对于初步筛选出的表，使用`table_details` Tool 获取详细架构和样本数据，进一步确认字段的有效性和数据格式。
    注意："table_details" Tool 输入参数为表名列表，比如感兴趣的表名是"table1"和"table2"，则输入参数为["table1", "table2"]。
    4. **构建SQL查询**: 基于验证过的字段信息，构建一个逻辑清晰、针对性强的SQL查询语句。
    5. **输出**: 向第二个AI代理(QueryExecutor & ResultInterpreter)传递构建好的SQL查询语句。

    '''
    
    messages = [{"role": "system","content": role_setting}]
    messages.append({"role": "user","content": '用户的问题是：' + user_input})
    
    
    # model_name = "glm-4"
    # llm = ZhipuModel(api_key=api_key_zhipu, model=model_name, temperature=0.9,tools=tools)
    model_name = "qwen-plus" 
    llm = QwenModel(api_key=api_key_qwen, model=model_name, temperature=0.2,tools=tools)
    logger.info('AI模型已初始化：%s', model_name)
    total_usage = 0
    # 下面的循环核心是执行response = llm.call(messages)，如果出现错误，等待一段时间后再次尝试，最多尝试3次
    response = call_model(llm, messages)
    messages.append(response.message)
    total_usage += response.usage["total_tokens"]
    logger.info('AI模型已调用：%s', response.message['content'])

    
    max_loop_count = 10
    loop_count = 0
    while loop_count < max_loop_count and ("tool_calls" in response.message):
        tool_call = response.message["tool_calls"][0]
        args = tool_call["function"]["arguments"]
        logger.info('处理工具调用：%s', tool_call["function"]["name"])
        logger.info('工具调用参数：%s', args)

        
        # 这部分处理tool call，即不同的工具名调用不同的函数
        if tool_call["function"]["name"] == "tables":
            try:
                function_result = tables()
            except Exception as e:
                function_result = {"error": str(e)}
        elif tool_call["function"]["name"] == "table_details":
            try:
                function_result = table_details(**json.loads(args))
            except Exception as e:
                function_result = {"error": str(e)}

        
        # 添加tool call的结果到 messages序列里。
        messages.append({
            "role": "tool",
            "content": f"{json.dumps(function_result)}",
            "tool_call_id":tool_call['id']
        })
        
        logger.info('工具调用的结果是：%s', function_result)

        
        # 下面的循环核心是执行response = llm.call(messages)，如果出现错误，等待一段时间后再次尝试，最多尝试3次
        response = call_model(llm, messages)
        total_usage += response.usage["total_tokens"]
        messages.append(response.message)
        logger.info('AI模型已再次调用，响应是：%s', response.message['content'])
    logger.info('AI模型已完成，总共使用的token数：%s。模型名：%s。', total_usage, model_name)
    return response.message

In [32]:
result = QueryBuilder("项目面积最大的标是哪个")

In [20]:
print(result)

我正在努力获取表`tender_key_detail`的详细信息，但似乎我需要使用不同的方法来调用工具。在这个阶段，我无法确认表是否包含我们需要的字段。在这种情况下，我通常会假设表中有“area”和标识项目的关键字段，如“project_id”或“project_name”。然而，为了提供准确的SQL查询，我确实需要知道这些字段是否存在。如果可以，能否提供关于表`tender_key_detail`结构的更多信息，或者确认包含面积和项目标识的字段名称？
